In [1]:
import numpy as np
import pandas as pd
import math
from abc import ABC, abstractmethod
import os
os.add_dll_directory(r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\bin")
import ibatensor


In [2]:
def print_tensor_info(label, tensor):
    print(f"{label} shape:", tensor.shape)
    s = tensor.to_string()
    print(s)

In [3]:
CUDA = 1
learning_rate = ibatensor.Tensor(np.full((1,1,1,1), 0.00001, dtype=np.float32), 1)
mew = ibatensor.Tensor(np.full((1,1,1,1), 0.9, dtype=np.float32), 1)
tensor_neg_one = ibatensor.Tensor(np.full((1,1,1,1), -1, dtype=np.float32), 1)

class Layer(ABC):

    @abstractmethod
    def __init__(self):
        pass

    @abstractmethod
    def forward(self):
        pass

    @abstractmethod
    def backward(self):
        pass
class Linear(Layer):
    def __init__(self, indim : int, outdim : int):
        weights_initializer = np.random.rand(outdim, indim)
        bias_initializer = np.random.rand(outdim)

        weights_initializer.resize((1,1, indim, outdim))
        self.weights = ibatensor.Tensor(weights_initializer, CUDA)

        bias_initializer.resize((1,1,1, outdim))
        self.bias = ibatensor.Tensor(bias_initializer, CUDA)

        self.prev_update_weights = None
        self.prev_update_bias = None
        self.prev_input = None


    def forward(self, X):
        self.prev_input = X

        return (X @ self.weights).elem_wise_add(self.bias)

    def backward(self, sigma):
        grad_w =  self.prev_input.mat_transpose() @ sigma
        grad_b = ibatensor.bias_backwards(sigma)

        if(self.prev_update_weights == None):
            update_weights = grad_w.elem_wise_mult(learning_rate).elem_wise_mult(tensor_neg_one)
            self.weights = self.weights.elem_wise_sub(update_weights)
            update_bias = grad_b.elem_wise_mult(learning_rate).elem_wise_mult(learning_rate)
            self.bias = self.bias.elem_wise_sub(update_bias)

            self.prev_update_weights = update_weights
            self.prev_update_bias = update_bias
        else:
            update_weights = self.prev_update_weights.elem_wise_mult(mew).elem_wise_sub(grad_w.elem_wise_mult(learning_rate).elem_wise_mult(tensor_neg_one))
            self.weights = self.weights.elem_wise_sub(update_weights)
            update_bias = self.prev_update_weights.elem_wise_mult(mew).elem_wise_sub(grad_b.elem_wise_mult(learning_rate).elem_wise_mult(learning_rate))
            self.bias = self.bias.elem_wise_sub(update_bias)

            self.prev_update_weights = update_weights
            self.prev_update_bias = update_bias

        return sigma @ self.weights.mat_transpose()

class ReLU(Layer):
    def __init__(self):
        self.prev_input = None
        return

    def forward(self, X):
        out = X.relu()
        self.prev_input = X
        return out


    def backward(self, sigma):
        return ibatensor.relu_backwards(sigma, self.prev_input)

class output_layer(Layer):
    def __init__(self):
        self.prev_output = None
        return
    def forward(self, X):
        result = X.softmax()
        self.prev_output = result
        return X.softmax()

    def backward(self, Y):
        return self.prev_output.elem_wise_sub(Y)

In [4]:
X = np.random.uniform(-5, 5, 100000)
Y_1 = X <= 0
Y_2 = X > 0
Y_1_one_hot = np.full((100000), 0, dtype = np.float32)
Y_1_one_hot[Y_1] = 1
Y_2_one_hot = np.full((100000), 0, dtype = np.float32)
Y_2_one_hot[Y_2] = 1
Y_one_hot = np.stack((Y_1_one_hot, Y_2_one_hot), axis=1) 
Y_one_hot
Y_one_hot.resize((1,1, 1000, 2))
X.resize((1,1,1000,1))
X = ibatensor.Tensor(X, 1)
Y = ibatensor.Tensor(Y_one_hot, 1)

In [5]:
linear_1 = Linear(1, 10)
relu_1 = ReLU()
linear_2 = Linear(10, 2)
relu_2 = ReLU()
output_L = output_layer()

In [6]:
result_l_1 = linear_1.forward(X)
result_relu_1 = relu_1.forward(result_l_1)
result_linear_2 = linear_2.forward(result_relu_1)
result_relu_2 = relu_2.forward(result_linear_2)
result = output_L.forward(result_linear_2)
print_tensor_info("res", result)


res shape: [1, 1, 1000, 2]

=== N=0, C=0 ===
0.259244	0.740756	
0.158113	0.841887	
0.0767164	0.923284	
0.158031	0.841969	
0.261924	0.738076	
0.067636	0.932364	
0.600267	0.399733	
0.600267	0.399733	
0.600324	0.399676	
0.093165	0.906835	
0.188639	0.811361	
0.600267	0.399733	
0.256737	0.743263	
0.600267	0.399733	
0.600267	0.399733	
0.517174	0.482826	
0.745089	0.254911	
0.308211	0.691789	
0.684855	0.315145	
0.930511	0.0694887	
0.914176	0.0858237	
0.207966	0.792034	
0.656027	0.343973	
0.33566	0.66434	
0.119985	0.880014	
0.269208	0.730792	
0.361162	0.638838	
0.678394	0.321606	
0.185283	0.814717	
0.857935	0.142065	
0.373922	0.626078	
0.782888	0.217112	
0.741923	0.258077	
0.0936932	0.906307	
0.538157	0.461843	
0.956667	0.043333	
0.873704	0.126296	
0.148318	0.851682	
0.778355	0.221645	
0.124347	0.875653	
0.922583	0.0774172	
0.165648	0.834352	
0.366923	0.633077	
0.488595	0.511405	
0.947794	0.052206	
0.194396	0.805604	
0.833113	0.166887	
0.864656	0.135344	
0.558129	0.441871	
0.823224	0.176776	
0.

In [7]:
sigma_output = output_L.backward(Y)
relu_2_sigma = relu_2.backward(sigma_output)
linear_2_sigma = linear_2.backward(relu_2_sigma)
relu_1_sigma = relu_1.backward(linear_2_sigma)
linear_1_sigma = linear_1.backward(relu_1_sigma)

In [8]:
for i in range(1, 1000):
    result_l_1 = linear_1.forward(X)
    result_relu_1 = relu_1.forward(result_l_1)
    result_linear_2 = linear_2.forward(result_relu_1)
    result_relu_2 = relu_2.forward(result_linear_2)
    result = output_L.forward(result_linear_2)
    sigma_output = output_L.backward(Y)
    relu_2_sigma = relu_2.backward(sigma_output)
    linear_2_sigma = linear_2.backward(relu_2_sigma)
    relu_1_sigma = relu_1.backward(linear_2_sigma)
    linear_1_sigma = linear_1.backward(relu_1_sigma)

In [9]:
result_l_1 = linear_1.forward(X)
result_relu_1 = relu_1.forward(result_l_1)
result_linear_2 = linear_2.forward(result_relu_1)
result_relu_2 = relu_2.forward(result_linear_2)
result = output_L.forward(result_linear_2)
print_tensor_info("res", result)

res shape: [1, 1, 1000, 2]

=== N=0, C=0 ===
5.14679e-11	1	
2.27282e-15	1	
4.4765e-21	1	
2.2505e-15	1	
6.43905e-11	1	
5.0241e-22	1	
0.956373	0.0436266	
0.955409	0.0445909	
0.82761	0.17239	
1.36725e-19	1	
7.08049e-14	1	
0.994003	0.00599667	
4.16785e-11	1	
0.997588	0.00241222	
0.998016	0.00198411	
0.00585544	0.994145	
0.999997	3.1729e-06	
1.51858e-10	1	
0.999178	0.000821741	
1	1.28353e-18	
1	9.37172e-17	
7.51843e-15	1	
0.990584	0.00941639	
1.60617e-09	1	
3.46642e-20	1	
4.29636e-12	1	
1.3206e-08	1	
0.998565	0.00143517	
5.07904e-16	1	
1	3.91423e-12	
3.69653e-08	1	
1	6.18866e-08	
0.999996	4.33144e-06	
1.93254e-22	1	
0.0101292	0.989871	
1	1.08974e-22	
1	3.0645e-13	
3.40818e-18	1	
1	1.01653e-07	
7.43161e-20	1	
1	1.14254e-17	
3.97869e-17	1	
2.10587e-08	1	
0.000247239	0.999753	
1	4.2659e-21	
1.54289e-15	1	
1	1.38998e-10	
1	1.36294e-12	
0.0443594	0.955641	
1	5.11665e-10	
3.38488e-10	1	
1.35229e-05	0.999987	
1	2.72798e-15	
0.999966	3.39292e-05	
4.29743e-17	1	
1	2.0406e-15	
0.916991	0.0830086	
1.0

In [10]:
print_tensor_info("Y", Y)

Y shape: [1, 1, 1000, 2]

=== N=0, C=0 ===
0	1	
0	1	
0	1	
0	1	
0	1	
0	1	
1	0	
1	0	
1	0	
0	1	
0	1	
1	0	
0	1	
1	0	
1	0	
1	0	
1	0	
0	1	
1	0	
1	0	
1	0	
0	1	
1	0	
0	1	
0	1	
0	1	
0	1	
1	0	
0	1	
1	0	
0	1	
1	0	
1	0	
0	1	
0	1	
1	0	
1	0	
0	1	
1	0	
0	1	
1	0	
0	1	
0	1	
0	1	
1	0	
0	1	
1	0	
1	0	
0	1	
1	0	
0	1	
0	1	
1	0	
1	0	
0	1	
1	0	
1	0	
0	1	
0	1	
0	1	
0	1	
1	0	
0	1	
1	0	
0	1	
0	1	
1	0	
1	0	
1	0	
1	0	
0	1	
1	0	
1	0	
1	0	
0	1	
0	1	
0	1	
0	1	
1	0	
1	0	
0	1	
0	1	
1	0	
0	1	
0	1	
1	0	
0	1	
0	1	
1	0	
0	1	
0	1	
0	1	
1	0	
1	0	
1	0	
0	1	
0	1	
1	0	
0	1	
0	1	
1	0	
1	0	
0	1	
1	0	
0	1	
0	1	
0	1	
0	1	
1	0	
0	1	
1	0	
1	0	
0	1	
1	0	
0	1	
1	0	
1	0	
1	0	
0	1	
1	0	
0	1	
1	0	
0	1	
0	1	
0	1	
0	1	
1	0	
0	1	
1	0	
1	0	
0	1	
0	1	
1	0	
0	1	
0	1	
0	1	
1	0	
1	0	
0	1	
1	0	
1	0	
0	1	
0	1	
0	1	
0	1	
1	0	
1	0	
1	0	
0	1	
1	0	
1	0	
0	1	
1	0	
0	1	
1	0	
0	1	
1	0	
1	0	
0	1	
0	1	
0	1	
1	0	
1	0	
0	1	
0	1	
0	1	
0	1	
1	0	
0	1	
0	1	
0	1	
0	1	
1	0	
1	0	
0	1	
0	1	
0	1	
0	1	
1	0	
1	0	
1	0	
0	1	
1	0	
0	1	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	